# Training data

In [79]:
import pickle
import numpy as np
import random
from operator import add

In [80]:
f = open ('./char_embedding.pkl', 'rb')
dict_char = pickle.load(f)
f.close()

f = open ('./dataset/item_index_withEN.pkl', 'rb')
item_index = pickle.load(f)
f.close()

f = open ('./dataset/name_index_withEN.pkl', 'rb')
user_index = pickle.load(f)
f.close()

In [81]:
rating_mat = np.load('dataset/user_item_withEN_mat.npy')

In [82]:
print("dict_char", len(dict_char))
print("user_index", len(user_index))
print("item_index", len(item_index))
print("rating_mat", rating_mat.shape)

dict_char 21592
user_index 15251
item_index 32
rating_mat (15251, 32)


In [83]:
#dict_char['沈']
#user_list.remove('')
k, v = random.choice(list(dict_char.items()))
print(type(v))
#print(user_list)

<class 'numpy.ndarray'>


In [91]:
user_list = list(user_index.keys())
print(len(user_list))
user_mat = []
count = 0

for user in user_list:
    temp = []
    for char in user:
        try:
            temp.append(dict_char[char])
        except:
            count += 1
            k, v = random.choice(list(dict_char.items()))
            temp.append(v)
    #print(len(temp))
    if len(temp) == 2:
        user_emb = np.mean([temp[0], temp[1]], axis=0)
    elif len(temp) == 3:
        user_emb = np.mean([temp[0], temp[1], temp[2]], axis=0)
    elif len(temp) == 4:
        user_emb = np.mean([temp[0], temp[1], temp[2], temp[-1]], axis=0)
    else:
        print("index", user_list.index(user))
        #rating_mat = np.delete(rating_mat, [user_list.index(user)], axis=0)
        user_list.remove(user)
        del user_index[user]
        print(len(temp))
    
    user_mat.append(user_emb)

user_mat = np.array(user_mat)
print("missing char", count)
print("user_mat", user_mat.shape)
print("remain", len(user_list))
print("remain", rating_mat.shape)

15250
missing char 190
user_mat (15250, 256)
remain 15250
remain (15250, 32)


In [194]:
#item_shape = (256, 32)
item_mat = np.random.rand(256, 32)

In [195]:
num_user = 15250
num_item = 32

# Model

In [196]:
from keras import Model
import keras.backend as K
from keras.layers import Embedding,Reshape,Input,Dot, Dense
K.clear_session()

In [197]:
def create_model(num_user, num_item, k):
    input_user = Input(shape=(256,),dtype="float32")
    #model_uer = Embedding(num_user+1,k,input_length = 1)(input_uer)
    model_user = Dense(k, activation='relu')(input_user)
    model_user = Reshape((k,))(model_user)
    
    input_item = Input(shape=(32,),dtype="float32")
    model_item  = Embedding(num_item+1,k,input_length = 32)(input_item)
    model_item = Dense(256, activation='relu')(model_item)
    model_item = Reshape((256,))(model_item)
    
    out = Dot(1)([model_user,model_item])
    model = Model(inputs=[input_user,model_item], outputs=out)
    model.compile(loss='mse', optimizer='Adam')
    model.summary()
    return model

In [220]:
def Recmand_model(num_user,num_item,k):
    input_uer = Input(shape=[None,],dtype="float32")
    #model_uer = Embedding(num_user+1,k,input_length = 1)(input_uer)
    model_uer = Reshape((k,))(input_uer)
    
    input_item = Input(shape=[k,],dtype="float32")
    #model_item  = Embedding(num_item+1,k,input_length = 1)(input_item)
    #model_item = Reshape((k,))(input_item)
    
    out = Dot(1)([model_uer,input_item])
    model = Model(inputs=[input_uer,input_item], outputs=out)
    model.compile(loss='mse', optimizer='Adam')
    model.summary()
    return model

In [224]:
model = Recmand_model(num_user, num_item, 32)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
reshape_17 (Reshape)            (None, 32)           0           input_19[0][0]                   
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 32)           0                                            
__________________________________________________________________________________________________
dot_8 (Dot)                     (None, 1)            0           reshape_17[0][0]                 
                                                                 input_20[0][0]                   
Total para

In [225]:
train_x = [user_mat, item_mat]
train_y = rating_mat

In [226]:
model.fit(train_x,train_y,batch_size = 100,epochs =10)

ValueError: Error when checking target: expected dot_8 to have shape (1,) but got array with shape (32,)